# 🌮 Recommendation System Vibe Check

This notebook helps you sanity-check the ML pipeline:

1. Connects to your local DB
2. Picks a user
3. Generates their feed using `RecommendationService`
4. Prints the results



In [ ]:
import os
import sys
import asyncio

# Add project root to path so `src` imports work
sys.path.append(os.path.abspath(os.path.join('..')))

from sqlalchemy import select, delete
from sqlalchemy.ext.asyncio import create_async_engine, AsyncSession
from sqlalchemy.orm import sessionmaker

from src.conf.settings import settings
from src.db.models import User, MealReview, UserFeedItem, TriState, Swipe
from src.services.recommendation import RecommendationService

# Adjust DB URL so it hits localhost when running outside docker
DB_URL = settings.sqlalchemy_async_database_url
if "postgres:5432" in DB_URL:
    DB_URL = DB_URL.replace("postgres:5432", "localhost:5432")

print(f"Connecting to: {DB_URL}")
engine = create_async_engine(DB_URL, echo=False)
AsyncSessionLocal = sessionmaker(engine, class_=AsyncSession, expire_on_commit=False)



In [ ]:
async def run_test():
    async with AsyncSessionLocal() as session:
        # Grab any user
        result = await session.execute(select(User).limit(1))
        user = result.scalars().first()

        if not user:
            print("❌ No users in DB. Seed some data first!")
            return

        print(f"🧑‍🍳 Running recs for {user.first_name or 'Unknown'} ({user.id})")

        # Clean slate
        await session.execute(delete(UserFeedItem).where(UserFeedItem.user_id == user.id))
        await session.commit()

        # Fire the service
        service = RecommendationService(session)
        await service.generate_feed(user.id)

        # Fetch results
        feed_result = await session.execute(
            select(UserFeedItem, MealReview)
            .join(MealReview, UserFeedItem.meal_review_id == MealReview.id)
            .where(UserFeedItem.user_id == user.id)
            .order_by(UserFeedItem.score.desc())
        )
        rows = feed_result.all()

        if not rows:
            print("⚠️ No recommendations generated. Need more reviews/swipes in DB.")
            return

        print(f"\n✅ Generated {len(rows)} recs:\n")
        print(f"{'SCORE':<8} | {'MEAL':<35} | TAGS")
        print('-' * 70)
        for item, review in rows:
            tags = []
            if review.is_spicy == TriState.yes:
                tags.append("🌶 spicy")
            if review.is_vegan == TriState.yes:
                tags.append("🥦 vegan")
            print(f"{item.score:<8.3f} | {review.meal_name[:33]:<35} | {', '.join(tags)}")

# Actually run it (Notebook friendly)
await run_test()

